# Implementing a Finite Algebra in Python

In the discussion and code to follow, **ALL** elements of an algebra will be implemented as Python strings, usually lowercase. We could implement elements as classes/objects themselves, but that would be inefficient. Recall, that ALL of a finite algebra's properties can be derived from its Cayley table, so we will focus on creating an efficient Cayley table implementation.

We will start the implementation by working with a simple finite algebra, Rock-Paper-Scissors.

Here are its elements, as a list of strings, and its Cayley table, as a list of lists of strings.

In [1]:
>>> rps_elements = ['r', 'p', 's']

In [2]:
>>> rps_table =[['r', 'p', 'r'],
>>>             ['p', 'p', 's'],
>>>             ['r', 's', 's']]

Using the *list* method, *index*, we can obtain the 0-based index of an element in the list of elements. For RPS, an index will be one of 0, 1, or 2.

Here are some examples:

In [3]:
>>> print(f"{rps_elements.index('r') = }")
>>> print(f"{rps_elements.index('p') = }")
>>> print(f"{rps_elements.index('s') = }")

rps_elements.index('r') = 0
rps_elements.index('p') = 1
rps_elements.index('s') = 2


Since the table is also a list (of lists), we can retrieve one of its rows by index as follows.

In [4]:
>>> print(f"{rps_table[0] = }")  # Row 0 (the first row)

rps_table[0] = ['r', 'p', 'r']


And then, we can add another index reference to the row to retrieve a specific (column) element as shown below.

In [5]:
>>> print(f"{rps_table[0][2] = }")  # Element in Row 0 and Col 2

rps_table[0][2] = 'r'


### Cayley Table of Indices

Representing an algebra's element list and Cayley table with strings is useful as a human readable representation, but for internal use, there are too many redundant strings. The only strings we really need to keep are the those in the list of elements. And, as long as we keep the elements in the element list in order, the ones in the table can be replaced with the integer-valued indices of the respective strings in the list of elements

Consider the first row of the table.

In [6]:
>>> rps_table[0]

['r', 'p', 'r']

It can be converted into indices corresponding to the element list using a list comprehension.

In [7]:
>>> [rps_elements.index(elem) for elem in rps_table[0]]

[0, 1, 0]

And, doing that for all of the table's rows provides us with a table of indices.

In [8]:
>>> [[rps_elements.index(elem) for elem in row] for row in rps_table]

[[0, 1, 0], [1, 1, 2], [0, 2, 2]]

Now, put all of that together into a function.

In [9]:
def strings_to_indices(elements, table):
    """Convert a Cayley table of strings into one of integer indices."""
    return [[elements.index(elem) for elem in row] for row in table]

Here's an example usage.

In [10]:
>>> strings_to_indices(rps_elements, rps_table)

[[0, 1, 0], [1, 1, 2], [0, 2, 2]]

## Cayley Table

Right now an algebra's Cayley table is simply a list of lists, but as noted above, all of a finite algebra's properties can be determined from the Cayley table, so let's create a Cayley table class in order to have a place to put the functionality for determining those properties.

Also, let's use NumPy arrays rather than a list of list so that we can take advantage of the additional functionality provided by NumPy arrays.

In [11]:
import numpy as np

class CayleyTable:

    def __init__(self, list_of_lists_of_ints):
        tmp = np.array(list_of_lists_of_ints, dtype=int)
        nrows, ncols = tmp.shape
        if nrows == ncols:
            if (np.min(tmp) >= 0) and (np.max(tmp) < nrows):
                self._table = tmp
            else:
                raise Exception(f"All integers must be between 0 and {nrows - 1}, inclusive.")
        else:
            raise Exception(f"Input arrays must be square; this one is {nrows}x{ncols}.")

    def __repr__(self):
        return f"{self.__class__.__name__}({self._table.tolist()})"

    def __getitem__(self, tup):
        row, col = tup
        return self._table[row][col]

    def __hash__(self):
        return hash(tuple(self._table.tolist()))

    @property
    def table(self):
        return self._table

    def tolist(self):
        return self._table.tolist()

    @property
    def order(self):
        return self._table.shape[0]

    def to_list_with_names(self, elements):
        return [[elements[index] for index in row] for row in self._table]

In [12]:
rps_ct = CayleyTable(strings_to_indices(rps_elements, rps_table))
rps_ct

CayleyTable([[0, 1, 0], [1, 1, 2], [0, 2, 2]])

In [13]:
rps_elements[rps_ct[0, 2]]

'r'

In [14]:
rps_ct.tolist()

[[0, 1, 0], [1, 1, 2], [0, 2, 2]]

In [15]:
rps_ct.to_list_with_names(rps_elements)

[['r', 'p', 'r'], ['p', 'p', 's'], ['r', 's', 's']]

In [16]:
rps_ct.to_list_with_names(['Rock', 'Paper', 'Scissors'])

[['Rock', 'Paper', 'Rock'],
 ['Paper', 'Paper', 'Scissors'],
 ['Rock', 'Scissors', 'Scissors']]

### Implementing a Binary Operator

Put all this together, we implement a *binary operator* as a **callable** Python class.

In [17]:
class BinaryOperator:

    def __init__(self, elements, table):
        self._elements = elements
        self._table = None

        if isinstance(table, CayleyTable):
            self._table = table
        if isinstance(table[0][0], str):
            self._table = CayleyTable(strings_to_indices(elements, table))
        elif isinstance(table[0][0], int):
            self._table = CayleyTable(elements, table)
        else:
            raise TypeError("Input table must a CayleyTable or a list-of-lists of str/int")

    def __call__(self, elem1, elem2):
        row_index = self._elements.index(elem1)
        col_index = self._elements.index(elem2)
        table_index = self._table[row_index, col_index]
        return self._elements[table_index]

    @property
    def cayley_table(self):
        return self._table

Here are some usage examples for the RPS binary operator:

In [18]:
rps_bin_op = BinaryOperator(rps_elements, rps_table)

print(f"{rps_bin_op('r', 'p') = }")  # r * p = p
print(f"{rps_bin_op('p', 'p') = }")  # p * p = p
print(f"{rps_bin_op('s', 'p') = }")  # s * p = s

rps_bin_op('r', 'p') = 'p'
rps_bin_op('p', 'p') = 'p'
rps_bin_op('s', 'p') = 's'


### Implementing a Finite Algebra

In [19]:
from abc import ABC, abstractmethod, abstractproperty

class FiniteAlgebra(ABC):

    def __init__(self, name, description, elements, table):
        """
        name:        A short string name of the algebra.
        description: A longer string describing the algebra.
        elements:    A list of strings representing the algebra's elements.
        table:       Cayley table in list-of-list of str or int, or an instance
                     of a CayleyTable object.
        """
        self._name = name
        self._description = description
        self._elements = elements
        self._op = BinaryOperator(elements, table)

    @property
    def name(self):
        return self._name

    @property
    def description(self):
        return self._description

    def elements(self):
        """Return the list of the algebra's elements"""
        return self._elements

    def op(self, elem1, elem2):
        """Applies the algebra's binary operator to two elements and returns the result."""
        return self._op(elem1, elem2)

    def __contains__(self, element):
        return element in self._elements

    def __getitem__(self, index):
        return self._elements[index]

    def __repr__(self):
        nm, desc, elems, tbl = self._name, self._description, self._elements, self._op.cayley_table.table
        return f"{self.__class__.__name__}(\n'{nm}',\n'{desc}',\n{elems},\n{tbl}\n)"

    def __str__(self):
        return f"<{self.__class__.__name__}:{self.name}, ID:{id(self)}>"

In [20]:
rps = FiniteAlgebra("RPS", "Algebra of Rock-Paper-Scissors", rps_elements, rps_table)

In [21]:
rps

FiniteAlgebra(
'RPS',
'Algebra of Rock-Paper-Scissors',
['r', 'p', 's'],
[[0 1 0]
 [1 1 2]
 [0 2 2]]
)

In [22]:
rps.op('r', 'p')

'p'